# Censo de poblacion y vivienda 2020

## Librerias y configuraciones

In [22]:
import os
import pandas as pd
import random as rnd
import numpy as np

In [23]:
# Variable definida para imprimir información adicional en la libreta.
verbose = True

## Importar archivos

In [24]:
# DATA_DIR es una variable de entorno en mi computadora a la cual se le 
# asignó el valor de el dicrectorio donde se guardan los datos a procesar.
data_dir = os.getenv('DATA_DIR')

# La información censal la guardo en una carpeta llamada EventosCensales 
# dentro del directorio de datos.
censales_dir = 'EventosCensales'

# Se respetó la estructura de carpetas original que se obtiene al descomprimir 
# el archivo zip que se obtiene de INEGI. Algunas veces se modifican estos 
# archivos asi que pueden cambiar. En este caso el zip se descomprimió en una 
# carpeta llamada iter_00_cpv2020.
censo_2020_dir = 'iter_00_cpv2020'

# Los datos se encuentran dentro de la carpeta conjunto_de_datos y el descriptor 
# de cada uno de los campos se encuentra en diccionario_datos.
subdir_datos = 'conjunto_de_datos'
subdir_descriptor = 'diccionario_datos'

# El nombre del archivo donde se encentran los datos es 
# 'conjunto_de_datos_iter_00CSV20.csv' y descriptor de la base tiene el nombre 
# 'diccionario_datos_iter_00CSV20.csv'. Los dos vienen en formato csv.
archivo_datos = 'conjunto_de_datos_iter_00CSV20.csv'
archivo_descriptor = 'diccionario_datos_iter_00CSV20.csv'

# Concatenar las rutas de archivos
ruta_de_archivo = os.path.join(data_dir, 
                               censales_dir, 
                               censo_2020_dir, 
                               subdir_datos, 
                               archivo_datos)

ruta_de_descriptor = os.path.join(data_dir, 
                                  censales_dir, 
                                  censo_2020_dir, 
                                  subdir_descriptor, 
                                  archivo_descriptor)

if verbose:

    print(f'El archivo de datos existe: {os.path.isfile(ruta_de_archivo)}')
    print(f'El descriptor de base de datos existe: {os.path.isfile(ruta_de_descriptor)}')


El archivo de datos existe: True
El descriptor de base de datos existe: True


In [25]:
# Importar archivo de datos
raw_censo_2020 = pd.read_csv(ruta_de_archivo)

C:\Users\Lorrain\AppData\Local\Temp\ipykernel_4700\2128926544.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_censo_2020 = pd.read_csv(ruta_de_archivo)


## Análisis inicial

La mayor parte de las columnas se importa como objeto, lo que significa que existen diferentes tipos de datos dentro de cada una de las columnas. Para homogeneizar los datos se tiene que hacer un analisis mas detallado para definir que tipo de dato debe de ser cada una de las columnas y cuales son los valores que no son de este tipo en cada una de ellas.

In [38]:
# Primera desc
raw_censo_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 286 entries, ENTIDAD to TAMLOC
dtypes: int64(6), object(280)
memory usage: 426.9+ MB


El primer registro es un buen acercamiento al tipo de dato que contiene cada una de las columnas. Representa el total nacional, y como tal hay columnas que no aplican. Tal es el caso de las columnas que definen el nombre y clave del Estado, Municipio, y Localidad. No contienen tampoco longitud y latiud, estos datos solo estan presentes en los registros que representan localidades especificas.

In [39]:
# Diccionario cuyas llaves son el nombre de columna y valores del primer registro 
# del dataframe.
ejemplo_datos = {key : value for key, value in 
                zip(raw_censo_2020.columns, raw_censo_2020.iloc[0])}

if verbose:
    print(f'Numero de columnas {len(ejemplo_datos.items())}')

Numero de columnas 286


Existen tres tipos de columnas que deben estar en formato string. El primer tipo es el que contiene los nombres de los Estados, Municipios y Localidades. El segundo tipo contiene las claves de los diferentes niveles de entidad. Y el tercero contiene las coodenadas geograficas en formato de grados minutos y seguntos. Las columnas que deberian ser texto son las siguientes:

['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD', 'LATITUD']

El resto de las columnas contiene valores numericos. Casi todas tienen el formato de numeros enteros pero 6 que contienen promedios y una que contiene la relacion que existe entre hombres y mujeres de la delimitación geoestadistica. Estos campos contienen decimales por lo que deben de tener el formato Float. Las columnas con valores decimales son:

['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 'PROM_OCUP', 'PRO_OCUP_C']

El resto de las columnas contienen numeros enteros.

In [41]:
# En general las libretas jupyter no imprimen un texto de mas de ciertas lineas.
# para ver todos los registros hay que cambiar los valores dentro de los
# corchetes ([0:20]) de la linea comentada para recorrerlos todos. Hay 286 pares 
# llave: valor en el diccionario.

#for item in list(ejemplo_datos.items())[0:20]: print(item)  

In [74]:
# Todas las columnas
columnas_censo_2020 = list(raw_censo_2020.columns)

# Columnas de texto
columnas_string = ['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                   'LONGITUD', 'LATITUD']
# Columnas con valores decimales
columnas_decimal = ['REL_H_M', 'PROM_HNV', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M', 
                    'PROM_OCUP', 'PRO_OCUP_C']

# El resto de las columnas son numeros enteros
columnas_entero = [i for i in columnas_censo_2020 if i not in columnas_string + columnas_decimal]

In [95]:
valores_string = {}
for column in columnas_entero:
        valores_string[column] = list(raw_censo_2020[~raw_censo_2020[column]
                                                .astype('string')
                                                .str
                                                .isnumeric()]
                                                [column].unique())


In [96]:
valores_string

{'ALTITUD': [1878.0,
  1902.0,
  1861.0,
  1989.0,
  1892.0,
  1971.0,
  1871.0,
  1908.0,
  1981.0,
  1927.0,
  1909.0,
  1889.0,
  1860.0,
  1825.0,
  1939.0,
  2023.0,
  1988.0,
  1888.0,
  1854.0,
  1826.0,
  1974.0,
  1907.0,
  1998.0,
  1972.0,
  2003.0,
  1984.0,
  1851.0,
  1970.0,
  1842.0,
  1900.0,
  1766.0,
  1879.0,
  1968.0,
  1936.0,
  1913.0,
  1873.0,
  1941.0,
  1877.0,
  1980.0,
  1797.0,
  1847.0,
  1855.0,
  1853.0,
  1845.0,
  1958.0,
  2028.0,
  1933.0,
  1870.0,
  1995.0,
  1903.0,
  1863.0,
  1808.0,
  1834.0,
  1881.0,
  1865.0,
  1856.0,
  2043.0,
  1876.0,
  1786.0,
  1899.0,
  1872.0,
  1947.0,
  1992.0,
  1862.0,
  1798.0,
  1959.0,
  1867.0,
  1964.0,
  2001.0,
  1961.0,
  2007.0,
  1986.0,
  1840.0,
  1954.0,
  1846.0,
  1843.0,
  1983.0,
  2026.0,
  1890.0,
  1813.0,
  1868.0,
  1880.0,
  1990.0,
  1850.0,
  2006.0,
  1882.0,
  1997.0,
  1985.0,
  1973.0,
  1859.0,
  1940.0,
  1935.0,
  1953.0,
  1994.0,
  2010.0,
  1996.0,
  1852.0,
  2036.0,
  2009.0,

## Cambiar el formato a columnas

### Columnas de texto

In [ ]:
for string_column, keylength in zip(['ENTIDAD', 'MUN', 'LOC'], [2, 3, 4]):
    raw_censo_2020[string_column] = raw_censo_2020[string_column].astype('string').str.zfill(keylength)



In [17]:
#raw_censo_2020[raw_censo_2020.eq('00-2').any(1)]
#raw_censo_2020[raw_censo_2020.eq('N/D').any(1)]

raw_censo_2020.eq('N/D').any(1).value_counts()


C:\Users\Lorrain\AppData\Local\Temp\ipykernel_4700\1042993367.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  raw_censo_2020.eq('N/D').any(1).value_counts()


False    195510
True        152
dtype: int64

In [9]:

column_dtype = {}
for column in raw_censo_2020.columns:

    if column in columnas_string:
        column_dtype[column] = 'string'

    elif column in columnas_decimal:
        column_dtype[column] = 'Float64'

    else:
        column_dtype[column] = 'Int64'


In [18]:

for string_column in columnas_string:
    if string_column not in ['ENTIDAD', 'MUN', 'LOC']:
        raw_censo_2020[string_column] = raw_censo_2020[string_column].astype("string")

for numeric_column in raw_censo_2020.columns:

    if numeric_column not in columnas_string + ['ALTITUD']:
        
        raw_censo_2020[numeric_column].replace(to_replace = ['*', 'N/D'], value = np.nan, inplace = True)

        if numeric_column in columnas_decimal:
            raw_censo_2020[numeric_column] = raw_censo_2020[numeric_column].astype('Float64')
        
        else:
            raw_censo_2020[numeric_column] = raw_censo_2020[numeric_column].astype('Int64')





In [19]:
columnas_string + columnas_decimal

['ENTIDAD',
 'NOM_ENT',
 'MUN',
 'NOM_MUN',
 'LOC',
 'NOM_LOC',
 'LONGITUD',
 'LATITUD',
 'REL_H_M',
 'PROM_HNV',
 'GRAPROES',
 'GRAPROES_F',
 'GRAPROES_M',
 'PROM_OCUP',
 'PRO_OCUP_C']

In [20]:
#raw_censo_2020.iloc[:,0:8].info()
#raw_censo_2020.loc[:,columnas_decimal].info()
raw_censo_2020.loc[:,raw_censo_2020.columns.drop(columnas_decimal+columnas_string)].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195662 entries, 0 to 195661
Columns: 271 entries, ALTITUD to TAMLOC
dtypes: Int64(270), object(1)
memory usage: 454.9+ MB


In [9]:
descriptor.RANGOS.unique()

array(['00…32', 'Alfanumérico', '000…570', '0000…9999', 'Caracter',
       '0...999999999', '0…999999999', '0.,.999999999', '0..999999999',
       '0...99999999', '0… 999999999', '01..14'], dtype=object)

In [13]:
nuevos_nombres = {
    'Núm.':'NUM', 
    'Indicador':'INDICADOR',
    'Descripción':'DESCRIPCION',
    'Mnemónico':'NOMCOL',
    'Rangos':'RANGOS',
    'Longitud':'LONGITUD'
}

descriptor = pd.read_csv(ruta_de_descriptor, skiprows=4, usecols=list(nuevos_nombres.keys()))
descriptor.rename(columns=nuevos_nombres, inplace=True)

In [43]:
raw_censo_2020.columns

Index(['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 'LONGITUD',
       'LATITUD', 'ALTITUD', 'POBTOT',
       ...
       'VPH_CEL', 'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ',
       'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT', 'VPH_SINTIC', 'TAMLOC'],
      dtype='object', length=286)